# Image Anomaly Detection  using PyTorch and <br>Intel® Transfer Learning Tool API

This notebook demonstrates anomaly detection using the Transfer Learning Toolkit (TLT). It performs defect analysis with the MVTec dataset using PyTorch. The workflow uses a pretrained ResNet50 v1.5 model from torchvision.

## 1. Import dependencies and setup parameters

This notebook assumes that you have already followed the instructions to setup a PyTorch environment with all the dependencies required to run the notebook.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import PIL.Image as Image
import torch, torchvision
from torchvision.transforms.functional import InterpolationMode
import requests
from io import BytesIO

# tlt imports
from tlt.datasets import dataset_factory
from tlt.models import model_factory
from tlt.utils.file_utils import download_and_extract_tar_file, download_file

# Specify a directory for the dataset to be downloaded
dataset_dir = os.environ["DATASET_DIR"] if "DATASET_DIR" in os.environ else \
    os.path.join(os.environ["HOME"], "dataset")
     
# Specify a directory for output
output_dir = os.environ["OUTPUT_DIR"] if "OUTPUT_DIR" in os.environ else \
    os.path.join(os.environ["HOME"], "output")

print("Dataset directory:", dataset_dir)
print("Output directory:", output_dir)

## 2. Get the model

In this step, we use the model factory to get the desired model. The get_model function returns a model object that will later be used for training.

In [ ]:
model = model_factory.get_model(model_name="resnet50", framework="pytorch", use_case='anomaly_detection')

## 3. Get the dataset

To use [MVTec](https://www.mvtec.com/company/research/datasets/mvtec-ad) or your own image dataset for anomaly detection, your image files (`.jpg` or `.png`) should be arranged in one of two ways. 

### Method 1: Category Folders

Arrange them in folders in the root dataset directory like this:

```
hazelnut
  └── crack
  └── cut
  └── good
  └── hole
  └── print
```

<b>IMPORTANT:</b> There must be a subfolder named `good` and at least one other folder of defective examples. It does not matter what the names of the other folders are or how many there, as long as there is at least one. This would also be an acceptable Method 1 layout:

```
toothbrush
  └── defective
  └── good
```

TLT will encode all of the non-good images as "bad" and use the "good" images in the training set and a mix of good and bad images in the validation set.

### Method 2: Train & Test Folders with Category Subfolders

Arrange them in folders in the root dataset directory like this:

```
hazelnut
  └── train
      └── good
  └── test
      └── crack
      └── cut
      └── good
      └── hole
      └── print
```

When using this layout, TLT will use the exact defined split for train and validation subsets unless you use the `shuffle_split` method to re-shuffle and split up the "good" images with certain percentages. 

In [ ]:
# Select the subdirectory in dataset_dir to use
img_dir = os.path.join(dataset_dir, 'hazelnut')

In [ ]:
# The defects argument can be used to filter the validation set to use only a subset of defect types
dataset = dataset_factory.load_dataset(img_dir, 
                                       use_case='image_anomaly_detection', 
                                       framework="pytorch",
                                       defects=['crack', 'hole']
                                      )

print(dataset._dataset)
print("Class names:", str(dataset.class_names))
print("Defect names:", dataset.defect_names)

## 4. Prepare the dataset
Once you have your dataset, use the following cells to split and preprocess the data. We split them into training and validation subsets, then resize the images to match the selected model, and then batch the images. Pass in optional arguments to customize the [Resize](https://pytorch.org/vision/main/generated/torchvision.transforms.Resize.html) or [Normalize](https://pytorch.org/vision/main/generated/torchvision.transforms.Normalize.html) transforms.
Data augmentation can be applied by specifying the augmentations to be applied in add_aug parameter. Supported augmentations are given below:
1. hflip - RandomHorizontalFlip
2. rotate - RandomRotate

In [ ]:
# If using Method 1 layout, split the dataset into training and validation subsets
dataset.shuffle_split(train_pct=.75, val_pct=.25)

In [ ]:
# Preprocess with an image size that matches the model, batch size 32, and the desired interpolation method
batch_size = 32
dataset.preprocess(model.image_size, batch_size=batch_size, interpolation=InterpolationMode.LANCZOS)

## 5. Visualize samples from the dataset

We get a single batch from our training and validation subsets and visualize the images as a sanity check.

In [ ]:
def plot_images(images, labels, sup_title, predictions=None):
    plt.figure(figsize=(18,14))
    plt.subplots_adjust(hspace=0.5)
    for n in range(min(batch_size, 30)):
        plt.subplot(6,5,n+1)
        inp = images[n]
        inp = inp.numpy().transpose((1, 2, 0))
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        inp = std * inp + mean
        inp = np.clip(inp, 0, 1)
        plt.imshow(inp)
        if predictions:
            correct_prediction = labels[n] == predictions[n]
            color = "darkgreen" if correct_prediction else "crimson"
            title = predictions[n] if correct_prediction else "{}".format(predictions[n])
        else:
            good_sample = labels[n] == 'good'
            color = "darkgreen" if good_sample else "crimson"
            title = labels[n]
        plt.title(title, fontsize=14, color=color)
        plt.axis('off')
    _ = plt.suptitle(sup_title, fontsize=20)
    plt.show()

In [ ]:
# Plot some images from the training set
images, labels = dataset.get_batch()
labels = [dataset.class_names[id] for id in labels]
plot_images(images, labels, 'Training Samples')

In [ ]:
# Plot some images from the validation set
images, labels = dataset.get_batch(subset='validation')
labels = [dataset.class_names[id] for id in labels]
plot_images(images, labels, 'Validation Samples')

## 6. Training and Evaluation

This step calls the model's train function with the dataset that was just prepared. The training function will get the torchvision feature extractor for the user's desired layer and extract features from the training set. The extracted features are used to perform a [principal component analysis](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). With the `do_eval` parameter set to True by default, this step will also show how the model can be evaluated. The model's evaluate function returns the AUROC metric ([area under](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.auc.html) the [roc curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html)) calculated from the dataset's validation subset.

### Train Arguments

#### Required
-  **dataset** (ImageAnomalyDetectionDataset, required): Dataset to use when training the model
-  **output_dir** (str): Path to a writeable directory

#### Optional
-  **layer_name** (str): The layer name whose output is desired for the extracted features
-  **pooling** (str): Pooling to be applied on the extracted layer ('avg' or 'max') (default: 'avg')
-  **kernel_size** (int): Kernel size in the pooling layer (default: 2)
-  **pca_threshold** (float): Threshold to apply to PCA model (default: 0.99)

Note: refer to release documentation for an up-to-date list of train arguments and their current descriptions

In [ ]:
# Examine the model's layers and decide which to use for feature extraction
model.list_layers(verbose=False)
layer = 'layer3'

In [ ]:
components = model.train(dataset, output_dir, layer_name=layer, pooling='avg', kernel_size=2, pca_threshold=0.99)

## 7. Predict

Using the same batch of validation sample from above, get and view the model's predictions.

In [ ]:
predictions = model.predict(images, return_type='class', threshold=-42)

In [ ]:
plot_images(images, labels, 'Predictions', predictions=predictions)
print("Correct predictions are shown in green")
print("Incorrect predictions are shown in red")

accuracy = sum([1 if p==labels[i] else 0 for i, p in enumerate(predictions)])/len(predictions)
print("Accuracy: {}".format(accuracy))

## 8. Optional: Use the feature extractor on a different data loader

If your dataset has a different format/layout than the one supported by TLT or if you want to perform your own modeling with the extracted features, just call the TLT model's `extract_features()` method directly passing in a `torch.Tensor` instead of through the `train()` call using a TLT dataset. This will give you complete flexibility and enable advanced custom analyses. You can also use the model's `pca()` method directly with the output of extract_features, as demonstrated below.

The following cells do almost the same thing as the above workflow, but the custom dataloader duplicates the validation set to achieve a size of 17,000 samples.

In [ ]:
import torchvision.transforms as TF
from torchvision.transforms.functional import InterpolationMode
from sklearn.decomposition import PCA
from sklearn import metrics
from torch.utils.data import Dataset
from tqdm import tqdm

# Set required variables if you aren't using a TLT dataset
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]
im_size=224
batch_size = 32
layer='layer3'
pool='avg' #avg/max
kernel_size=2
pca_thresholds=0.99
device = "cpu"
num_workers=56

# Give the path and custom class for your dataset
object_type = 'hazelnut'
root_dir = img_dir = os.path.join(dataset_dir, 'mvtec')

class Mvtec(Dataset):
    def __init__(self, root_dir, object_type=None, split=None, defect_type=None, im_size=None, transform=None):
        
        if split == 'train':
            # defect_type = 'good'
            csv_name = '{}_train.csv'.format(object_type)
        else:
            csv_name = '{}_{}.csv'.format(object_type, defect_type)

        csv_file = os.path.join(root_dir, object_type, csv_name)
        # self.image_folder = os.path.join(root_dir, object_type, split, defect_type)
        self.data_frame = pd.read_csv(csv_file)
        self.image_dir = os.path.join(root_dir, object_type)
        if transform:
            self.transform = transform
        else:
            self.im_size = (224, 224) if im_size is None else (im_size, im_size)
            normalize_tf = TF.Normalize(mean=imagenet_mean, std=imagenet_std)
            self.transform = TF.Compose([TF.Resize(tuple(self.im_size), interpolation=InterpolationMode.LANCZOS), TF.ToTensor(), normalize_tf])
        self.num_classes = 1


    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name)
        if image.mode == 'L':
            image = image.convert('RGB')
        image = self.transform(image)
        labels = self.data_frame.iloc[idx, 1]
        sample = {'data': image, 'label': labels}

        return sample

    def getclasses(self):
        classes = [str(i) for i in range(self.num_classes)]
        c = dict()
        for i in range(len(classes)):
            c[i] = classes[i]
        return c

In [ ]:
print("Preparing Dataloader for Training Images \n")
trainset = Mvtec(root_dir,object_type=object_type,split='train',im_size=im_size)
testset = Mvtec(root_dir,object_type=object_type,split='test',defect_type='all',im_size=im_size)

classes = trainset.getclasses()
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

In [ ]:
print("Extracting features for", len(train_loader.dataset), "Training Images \n")
eval_loader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=False)
data = next(iter(eval_loader))
outputs_inner = model.extract_features(data['data'].to(device), layer, pooling=[pool, kernel_size])

In [ ]:
data_mats_orig = torch.empty((outputs_inner.shape[1], len(trainset))).to(device)
with torch.no_grad():
    data_idx = 0
    num_ims = 0
    for data in tqdm(train_loader):
        images = data['data']
        labels = data['label']
        images, labels = images.to(device), labels.to(device)
        num_samples = len(labels)
        
        outputs = model.extract_features(images, layer, pooling=[pool, kernel_size])
        
        oi = torch.squeeze(outputs)
        data_mats_orig[:, data_idx:data_idx+num_samples] = oi.transpose(1, 0)
        num_ims += 1
        data_idx += num_samples

In [ ]:
# PCA Modeling
print("Train PCA Kernel on training images \n")
pca_mats = model.pca(data_mats_orig)
features_reduced = pca_mats.transform(data_mats_orig.T)
features_reconstructed = pca_mats.inverse_transform(features_reduced)
print("Training Complete \n")

In [ ]:
print("Inference Evaluation Begins on", len(test_loader.dataset), "Test Images \n")

with torch.no_grad():
    len_dataset = len(test_loader.dataset)
    gt = torch.zeros(len_dataset)

    scores = np.empty(len_dataset)

    count = 0
    time_inf = 0
    time_score = 0
    for k, data in enumerate(tqdm(test_loader)):

        inputs = data['data'].to(memory_format=torch.channels_last)
        labels = data['label']
        num_im = inputs.shape[0]

        outputs = model.extract_features(inputs, layer, pooling=[pool, kernel_size])

        feature_shapes = outputs.shape
        oi = outputs
        oi_or = oi
        oi_j = pca_mats.transform(oi)
        oi_reconstructed = pca_mats.inverse_transform(oi_j)
        fre = torch.square(oi_or - oi_reconstructed).reshape(feature_shapes)
        fre_score = torch.sum(fre, dim=1)  # NxCxHxW --> NxHxW   
        scores[count: count + num_im] = -fre_score

        gt[count:count + num_im] = labels
        count += num_im

    gt = gt.numpy()

In [ ]:
print("AUROC is computed on", len_dataset, "Test Images \n")
fpr_binary, tpr_binary, _ = metrics.roc_curve(gt, scores)
auc_roc_binary = metrics.auc(fpr_binary, tpr_binary)
print(f'AUROC: {auc_roc_binary*100}')

## Dataset Citations

Paul Bergmann, Kilian Batzner, Michael Fauser, David Sattlegger, Carsten Steger: The MVTec Anomaly Detection Dataset: A Comprehensive Real-World Dataset for Unsupervised Anomaly Detection; in: International Journal of Computer Vision 129(4):1038-1059, 2021, DOI: 10.1007/s11263-020-01400-4.

Paul Bergmann, Michael Fauser, David Sattlegger, Carsten Steger: MVTec AD — A Comprehensive Real-World Dataset for Unsupervised Anomaly Detection; in: IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR), 9584-9592, 2019, DOI: 10.1109/CVPR.2019.00982.